In [1]:
from pandas import read_json
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
df = read_json("reddit_jokes.json")
sentences = df['title'] + ' ' + df['body']
# this analyzer from sklearn tokenizes including stop words and lower case
analyze = CountVectorizer(stop_words='english').build_analyzer()
sentences = [analyze(s) for s in sentences.tolist()]
# this trains a cbow on sentences
model = word2vec.Word2Vec(sentences, size=100, min_count=10, workers=4)
model.wv.save('reddit_jokes.model')

In [1]:
# Read the non-trainable part of the model from disk
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load('reddit_jokes.model')
google_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  



Here's some similarity comparisons between Google's massive model and ours.  Ours does surprisingly well

In [2]:
#w = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
#g = google_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
def Display(s):
    w = eval('word_vectors.' + s)
    g = eval('google_vectors.' + s)
    #import pdb; pdb.set_trace()
    print ('{}'.format(s))
    print('Trained by Jokes\tTrained by Google')
    for i in range(4):  #range(len(w)):
        print("{}\t{:4.2}\t\t{}\t{:4.2}".format(w[i][0], w[i][1], g[i][0], g[i][1]))
tests = ["most_similar(positive=['king'])", 
         "most_similar(positive=['king','woman'])",
         "most_similar(positive=['king','woman'], negative=['man'])",
         "most_similar(positive=['man'])",
         "most_similar(positive=['woman'])",
        ]
for s in tests:
    print('='*80)
    Display(s)

most_similar(positive=['king'])
Trained by Jokes	Trained by Google
queen	0.74		kings	0.71
arthur	0.72		queen	0.65
lancelot	0.65		monarch	0.64
kings	0.65		crown_prince	0.62
most_similar(positive=['king','woman'])
Trained by Jokes	Trained by Google
queen	0.63		man	0.66
arthur	0.57		queen	0.64
beauty	0.55		girl	0.61
bodyguard	0.53		princess	0.61
most_similar(positive=['king','woman'], negative=['man'])
Trained by Jokes	Trained by Google
queen	0.65		queen	0.71
elizabeth	0.59		monarch	0.62
birbal	0.57		princess	0.59
arthur	0.55		crown_prince	0.55
most_similar(positive=['man'])
Trained by Jokes	Trained by Google
mans	0.67		woman	0.77
gentleman	0.66		boy	0.68
guy	0.66		teenager	0.66
woman	0.62		teenage_girl	0.61
most_similar(positive=['woman'])
Trained by Jokes	Trained by Google
lady	0.75		man	0.77
man	0.62		girl	0.75
women	0.62		teenage_girl	0.73
gentleman	0.58		teenager	0.63


In [3]:
# Show me the vectorization of the word 'king'
vKing = word_vectors['king']
print ('Length of our vectorization of "king" (compare to the ctor for Word2Vec): {}'.format(len(vKing)))
print ('First 12 features of "king":  {}'.format(vKing[:12]))
# Show me the vocabulary itself# Export to CSV?  Real close here but I don't think we want to go this way.
# Instead, use the gensim API / "KeyedVector" (word_vector here).
from __future__ import print_function


print(type(word_vectors))
dump_embedding()
print ('The vocabulary is stored as a standard {} of len {}'.format(type(word_vectors.vocab), len(word_vectors.vocab)))
print ('There is also a {}, also of len {}'.format(type(word_vectors.index2word), len(word_vectors.index2word)))
print ('Here are the first 10 entries:  {}'.format(word_vectors.index2word[:10]))

Length of our vectorization of "king" (compare to the ctor for Word2Vec): 100
First 12 features of "king":  [-1.53627563 -1.44087207  0.02402437  1.61712003 -0.89577299 -0.04609798
 -1.54079127  1.01438117  0.84849453  0.80059016  1.14108241 -1.77830291]
<class 'gensim.models.keyedvectors.KeyedVectors'>
man, 1.98347747326, 1.96043431759, 
says, 1.17057991028, 0.387207120657, 
said, 1.29437792301, -1.11761200428, 
did, -0.731746554375, -1.54007816315, 
like, 0.520845174789, -0.422606885433, 
just, 0.421174943447, 2.01930809021, 
don, -0.217230424285, 1.85953974724, 
know, 2.2788233757, 0.290828645229, 
guy, 2.4918589592, 2.01255893707, 
asks, 2.19117808342, 0.256881028414, 
The vocabulary is stored as a standard <type 'dict'> of len 20012
There is also a <type 'list'>, also of len 20012
Here are the first 10 entries:  [u'man', u'says', u'said', u'did', u'like', u'just', u'don', u'know', u'guy', u'asks']


In [4]:
femininity = word_vectors['king'] - word_vectors['queen']
near_woman = word_vectors['man'] - femininity
print('Joke-trained, near woman?')
print(word_vectors.most_similar(positive=[near_woman]))

femininity = google_vectors['king'] - google_vectors['queen']
near_woman = google_vectors['man'] - femininity
print('Google-trained, near woman?')
print(google_vectors.most_similar(positive=[near_woman]))


Joke-trained, near woman?
[(u'man', 0.8185614347457886), (u'gentleman', 0.5897939205169678), (u'woman', 0.5865741968154907), (u'mans', 0.5807508230209351), (u'lady', 0.5801395177841187), (u'businessman', 0.5530569553375244), (u'guy', 0.5443193912506104), (u'husband', 0.4750097990036011), (u'madam', 0.47310683131217957), (u'smiles', 0.4672502279281616)]
Google-trained, near woman?
[(u'woman', 0.718680202960968), (u'man', 0.6557512283325195), (u'girl', 0.5882835388183594), (u'lady', 0.5754351615905762), (u'teenage_girl', 0.5700528621673584), (u'teenager', 0.5378326177597046), (u'schoolgirl', 0.497780978679657), (u'policewoman', 0.49065014719963074), (u'blonde', 0.4870774447917938), (u'redhead', 0.4778464436531067)]


In [1]:
from __future__ import print_function
import numpy as np
from pandas import read_json
from sklearn.feature_extraction.text import CountVectorizer

def doc2avgcbow(sentence):
    # this analyzer from sklearn tokenizes including stop words and lower case
    analyze = CountVectorizer(stop_words='english').build_analyzer()
    tokens = analyze(sentence)
    avg_cbow = np.zeros(100)
    N = 0
    for token in tokens:
        #print('   {}'.format(token))
        if token in word_vectors.vocab:
            N += 1
            avg_cbow += word_vectors[token]
    if N != 0:
        # Some jokes contain nothing in our vocabulary.  Examples.
        #  What do you call a shart? Woopsie Poopsie :) 
        # 1 2 3 4 5 6
        # Who do you call when a sleepwalker injures himself? The somnambulance.
        avg_cbow = avg_cbow/float(N)
    return avg_cbow

def avg_cbow():
    sentences = df['title'] + ' ' + df['body']
    for s in sentences:
        yield doc2avgcbow(s)
df = read_json("reddit_jokes.json")
df['cbow'] = [f for f in avg_cbow()]


NameError: global name 'word_vectors' is not defined

In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
X = df['cbow']
# our EDA revealed that a score of > 250 is about the top 5% of all scores.
# we'll train to find those.
labels = ['meh', 'liked']
y = pd.cut(df.score, [0, 250, 100000], include_lowest=True, labels=labels)
#y = pd.cut(df.score, 3, labels=['low', 'med', 'high'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
for clf, name in [
                  (GaussianNB(), "Naive Bayes"), 
                  (DecisionTreeClassifier(), "DecisionTree"), #fast
                  (SVC(kernel="linear", C=0.025), "SVC(kernel='linear', C=0.025)"), #high score, long time
                  (SVC(gamma=2, C=1), "SVC(gamma=2, C=1)") # very long time.
                 ]:
    clf.fit(X_train.tolist(), y_train.tolist())
    score = clf.score(X_test.tolist(), y_test.tolist())
    y_pred = clf.predict(X_test.tolist())
    con_mat = confusion_matrix(y_test, y_pred, labels=labels)
    print('{}, score {}'.format(name, score))
    for l in [' '] + labels:
        print('{:7}'.format(l), end='')
    print('')
    for i,r in enumerate(con_mat):
        print('{:7}'.format(labels[i]), end='')
        for e in r:
            print('{:7}'.format(e), end='')
        print('')    
    print(classification_report(y_test, y_pred, target_names=labels))



Naive Bayes, score 0.669373698954
       meh    liked  
meh      24967  11915
liked      950   1079
             precision    recall  f1-score   support

        meh       0.08      0.53      0.14      2029
      liked       0.96      0.68      0.80     36882

avg / total       0.92      0.67      0.76     38911

DecisionTree, score 0.890442291383
       meh    liked  
meh      34390   2492
liked     1771    258
             precision    recall  f1-score   support

        meh       0.09      0.13      0.11      2029
      liked       0.95      0.93      0.94     36882

avg / total       0.91      0.89      0.90     38911

SVC(kernel='linear', C=0.025), score 0.947855362237
       meh    liked  
meh      36882      0
liked     2029      0
             precision    recall  f1-score   support

        meh       0.00      0.00      0.00      2029
      liked       0.95      1.00      0.97     36882

avg / total       0.90      0.95      0.92     38911



/home/msr/.virtualenvs/lab/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC(gamma=2, C=1), score 0.946133484105
       meh    liked  
meh      36738    144
liked     1952     77
             precision    recall  f1-score   support

        meh       0.35      0.04      0.07      2029
      liked       0.95      1.00      0.97     36882

avg / total       0.92      0.95      0.93     38911

